In [ ]:
# ライブラリの読み込み
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Bidirectional, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

np.random.seed(123)
tf.random.set_seed(123)

In [ ]:
# データを作成する
def sin(x, T=100):
    return np.sin(2.0 * np.pi * x / T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2*T + 1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

T = 100
f = toy_problem(T).astype(np.float32)
length_of_sequences = len(f)

In [ ]:
# データの確認
plt.plot(range(length_of_sequences), f)
plt.show()

In [ ]:
# 教師データと評価データを作成
maxlen = 25

x = []
t = []

for i in range(length_of_sequences - maxlen):
    x.append(f[i:i+maxlen])
    t.append(f[i+maxlen])

x = np.array(x).reshape(-1, maxlen, 1)
t = np.array(t).reshape(-1, 1)

x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.2, shuffle=True)

In [ ]:
# モデルの構築
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='tanh',
               recurrent_activation='sigmoid',
               kernel_initializer='glorot_normal',
               recurrent_initializer='orthogonal')))
model.add(Dense(1, activation='linear'))
optimizer = optimizers.Adam(learning_rate=0.001,
                                beta_1=0.9, beta_2=0.999, amsgrad=True)

model.compile(optimizer=optimizer,
                  loss='mean_squared_error')

In [ ]:
#学習
es = EarlyStopping(monitor='val_loss',
                       patience=10,
                       mode="auto")
hist = model.fit(x_train, t_train,
                     epochs=1000, batch_size=100,
                     verbose=2,
                     validation_data=(x_val, t_val),
                     callbacks=[es])

In [ ]:
# モデルの評価
sin = toy_problem(T, ampl=0.)
gen = [None for i in range(maxlen)]

z = x[:1]

# 逐次的に予測値を求める
for i in range(length_of_sequences - maxlen):
    preds = model.predict(z[-1:])
    z = np.append(z, preds)[1:]
    z = z.reshape(-1, maxlen, 1)
    gen.append(preds[0, 0])

# 予測値を可視化
plt.rc('font', family='serif')
plt.xlim([0, 2*T])
plt.ylim([-1.5, 1.5])
plt.plot(range(len(f)), sin,
         color='gray',
         linestyle='--', linewidth=0.5)
plt.plot(range(len(f)), gen,
         color='black', linewidth=1,
         marker='o', markersize=1, markerfacecolor='black',
         markeredgecolor='black')
plt.show()